# Using Quara's standard tomography features from Qiskit

Quara supports the wrappers for executing standard tomography from qiskit. Here, we demonstarate how to use them as several 1-qubit tomography.

In [1]:
import numpy as np

from quara.interface.qiskit.api import (
    estimate_standard_qst_from_qiskit,
    estimate_standard_povmt_from_qiskit,
    estimate_standard_qpt_from_qiskit,
    generate_empi_dists_from_qiskit_state,
    generate_empi_dists_from_qiskit_povm,
    generate_empi_dists_from_qiskit_gate,
    generate_empi_dists_from_quara
) 

from quara.objects.state_typical import (
    generate_state_density_mat_from_name,
    get_state_names_1qubit,
    generate_state_from_name,
)

from qiskit.circuit.library.standard_gates import h
from qiskit.quantum_info.operators.channel import Choi

When executing the tomography in Qiskit, state and POVM of testers are defined as TomographyBasis. Here,we use following representation matrices from TomographyBasis(PauliBasis).

In [2]:
# State
Xp = np.array([[0.5, 0.5], [0.5, 0.5]], dtype=complex)
Xm = np.array([[0.5,-0.5],[-0.5,0.5]],dtype=complex)
Yp = np.array([[0.5, -0.5j], [0.5j, 0.5]], dtype=complex)
Ym = np.array([[0.5,0.5j],[-0.5j,0.5]],dtype=complex)
Zp = np.array([[1, 0], [0, 0]], dtype=complex)
Zm = np.array([[0, 0], [0, 1]], dtype=complex)
a = generate_state_density_mat_from_name("a")

#POVM
X = [Xp,Xm]
Y = [Yp,Ym]
Z = [Zp,Zm]

## Quantum State Tomography(1-qubit)
We demonstarate 1-qubit QST using the tester from Qiskit. We need to define tester POVMs as a list when we perform Quantum State tomography using quara.

In [3]:
#Testers
tester_povms = [X,Y,Z]
#True objects
true_state = a
print(true_state)

[[0.5       +0.j         0.35355339-0.35355339j]
 [0.35355339+0.35355339j 0.5       +0.j        ]]


We will generate the empirical distribution from given true state object. If you have your own list of empirical distributions make sure that the type matches to `List[Tuple [int,np.ndarray]]`.

In [18]:
# define system
mode = "qubit"
num = 1

# define requirement for empirical distribution
num_data = 1000
seed = 7896

# calculate empirical distribution
empi_dists = generate_empi_dists_from_qiskit_state(mode_system=mode, num_system=num, true_state=true_state, tester_povms=tester_povms, num_sum=num_data, seed=seed, schedules="all")
for f in empi_dists:
    print(f)
type(empi_dists)

(1000, array([0.864, 0.136]))
(1000, array([0.844, 0.156]))
(1000, array([0.49, 0.51]))


list

Now, we execute quantum state tomography using linear estimation. You can estimate the quantum state just by calling `estimate_standard_qst_from_qiskit()`. You can choose linear estimation by specifying `estimator_name="linear"`.
You have to specify the empi_dists in qiskit.

In [6]:
empi_dists_qiskit = generate_empi_dists_from_quara(empi_dists)
estimate = estimate_standard_qst_from_qiskit(mode_system=mode,num_system=num,tester_povms=tester_povms,empi_dists=empi_dists_qiskit[1],shots=1000,label=empi_dists_qiskit[0],estimator_name="linear", schedules="all")
print(estimate)

[[0.49 +0.j    0.364-0.344j]
 [0.364+0.344j 0.51 +0.j   ]]


You can choose least squares estimation by specifying `estimator_name="least_squares"`.

In [8]:
empi_dists_qiskit = generate_empi_dists_from_quara(empi_dists)
estimate = estimate_standard_qst_from_qiskit(mode_system=mode,num_system=num,tester_povms=tester_povms,empi_dists=empi_dists_qiskit[1],shots=1000,label=empi_dists_qiskit[0],estimator_name="least_squares", schedules="all")
print(estimate)

[[0.49001861+0.j         0.36332341-0.34336059j]
 [0.36332341+0.34336059j 0.50998144+0.j        ]]


## Quantum POVM Tomography (1qubit)

Here, we demonstarate 1-qubit POVMT using the tester from Qiskit. We need to define tester states as a list when we perform Quantum POVM tomography using quara.

In [9]:
#Testers
tester_states = [Xp,Yp,Zp,Zm]
#True objects
true_povm = X
for i in true_povm:
    print(i)

[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]


In [10]:
# define system
mode = "qubit"
num = 1

# define requirement for empirical distribution
num_data = 1000
seed = 7896

# calculate empirical distribution
empi_dists = generate_empi_dists_from_qiskit_povm(mode_system=mode, num_system=num, true_povm=true_povm, tester_states=tester_states, num_sum=num_data, seed=seed, schedules="all")
for i in empi_dists:
    print(i)

(1000, array([1., 0.]))
(1000, array([0.502, 0.498]))
(1000, array([0.49, 0.51]))
(1000, array([0.488, 0.512]))


Now, we execute quantum povm tomography using linear estimation. You can estimate the quantum povm just by calling `estimate_standard_povmt_from_qiskit()`. You can choose linear estimation by specifying `estimator_name="linear"`.
You have to specify num_outcomes, which is number of outcome values of the POVM that will be estimated.

In [11]:
empi_dists_qiskit = generate_empi_dists_from_quara(empi_dists)
estimate = estimate_standard_povmt_from_qiskit(mode_system=mode,num_system=num,tester_states=tester_states,empi_dists=empi_dists_qiskit[1],shots=1000,label=empi_dists_qiskit[0],num_outcomes=2,estimator_name="linear", schedules="all")
for i in estimate:
    print(i)

[[0.49 +0.j    0.511-0.013j]
 [0.511+0.013j 0.488+0.j   ]]
[[ 0.51 +0.j    -0.511+0.013j]
 [-0.511-0.013j  0.512+0.j   ]]


You can choose least squares estimation by specifying `estimator_name="least_squares"`.

In [12]:
empi_dists_qiskit = generate_empi_dists_from_quara(empi_dists)
estimate = estimate_standard_povmt_from_qiskit(mode_system=mode,num_system=num,tester_states=tester_states,empi_dists=empi_dists_qiskit[1],shots=1000,label=empi_dists_qiskit[0],num_outcomes=2,estimator_name="least_squares", schedules="all")
for i in estimate:
    print(i)

[[0.49735047+0.j         0.49632565-0.00556025j]
 [0.49632565+0.00556025j 0.4953651 +0.j        ]]
[[ 0.50264953+0.j         -0.49632565+0.00556025j]
 [-0.49632565-0.00556025j  0.5046349 +0.j        ]]


## Quantum Process Tomography(1qubit)

Here, we demonstarate 1-qubit QPT using the tester from Qiskit. We need to define tester states and POVMs as a list when we perform Quantum Process tomography using quara.  When using gate here, we have to use it in Choi matrix.

In [14]:
#Testers
tester_povms = [X,Y,Z]
tester_states = [Xp,Yp,Zp,Zm]
#True objects
true_gate = h.HGate()
true_gate.__array__()

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [15]:
# define system
mode = "qubit"
num = 1

# define requirement for empirical distribution
num_data = 1000
seed = 7896

# calculate empirical distribution
t_gate = Choi(true_gate)
empi_dists = generate_empi_dists_from_qiskit_gate(mode_system=mode, num_system=num, true_gate=t_gate, tester_states=tester_states,tester_povms=tester_povms, num_sum=num_data, seed=seed, schedules="all")
for i in empi_dists:
    print(i)

(1000, array([0.496, 0.504]))
(1000, array([0.502, 0.498]))
(1000, array([1., 0.]))
(1000, array([0.488, 0.512]))
(1000, array([0., 1.]))
(1000, array([0.491, 0.509]))
(1000, array([1., 0.]))
(1000, array([0.507, 0.493]))
(1000, array([0.513, 0.487]))
(1000, array([0., 1.]))
(1000, array([0.47, 0.53]))
(1000, array([0.497, 0.503]))


Now, we execute quantum process tomography using linear estimation. You can estimate the quantum process just by calling `estimate_standard_qpt_from_qiskit()`. You can choose linear estimation by specifying `estimator_name="linear"`.


In [16]:
empi_dists_qiskit = generate_empi_dists_from_quara(empi_dists)
estimate = estimate_standard_qpt_from_qiskit(mode_system=mode,num_system=num,tester_states=tester_states,tester_povms=tester_povms,empi_dists=empi_dists_qiskit[1],shots=1000,label=empi_dists_qiskit[0],estimator_name="linear", schedules="all")
for i in estimate:
    print(i)

[ 0.513 +0.j      0.5   -0.007j   0.495 -0.014j  -0.4925-0.0255j]
[ 0.5   +0.007j   0.487 +0.j      0.4845+0.0015j -0.495 +0.014j ]
[ 0.495 +0.014j   0.4845-0.0015j  0.497 +0.j     -0.5   +0.03j  ]
[-0.4925+0.0255j -0.495 -0.014j  -0.5   -0.03j    0.503 +0.j    ]


You can choose least squares estimation by specifying `estimator_name="least_squares"`.

In [17]:
empi_dists_qiskit = generate_empi_dists_from_quara(empi_dists)
estimate = estimate_standard_qpt_from_qiskit(mode_system=mode,num_system=num,tester_states=tester_states,tester_povms=tester_povms,empi_dists=empi_dists_qiskit[1],shots=1000,label=empi_dists_qiskit[0],estimator_name="least_squares", schedules="all")
for i in estimate:
    print(i)

[ 0.51438824+0.j          0.49329968-0.00741707j  0.48853761-0.00725424j
 -0.49249075+0.00293811j]
[ 0.49329968+0.00741707j  0.48561176+0.j          0.48871849+0.0007454j
 -0.48853761+0.00725424j]
[ 0.48853761+0.00725424j  0.48871849-0.0007454j   0.49664456+0.j
 -0.49336835+0.01536569j]
[-0.49249075-0.00293811j -0.48853761-0.00725424j -0.49336835-0.01536569j
  0.50335544+0.j        ]
